In [19]:
import net as nn
import torch
import pydicom
from PIL import Image as im
from PIL import ImageDraw, ImageOps
import numpy as np
import cv2
from ipywidgets import *
import xnat
from ipycanvas import Canvas, hold_canvas

net = nn.Net()
net.load_weights('trained_data.npy')

def check_project(project):
    text_bboxes = []
    
    project_progress.layout = Layout(display = 'inherit')
    experiment_progress.layout = Layout(display = 'inherit')
    
    project_progress.max = len(project.experiments)
    project_progress.value = 0
    
    for experimentID in project.experiments:
        project_progress.value += 1
        
        experiment = project.experiments[experimentID]
        if 'SessionData' in experiment._XSI_TYPE:
            try:
                experiment_progress.max = len(experiment.scans)
                experiment_progress.value = 0
                
                for scanID in experiment.scans:
                    experiment_progress.value += 1
                    
                    scan = experiment.scans[scanID]
                                        
                    download_scan(scan, progress)
                                        
                    directory = 'TempImages'
                    images = get_pixel_data(directory)

                    progress.layout = Layout(display = 'none')

                    info_text.value = 'Checking for text...'
                    if len(images) > 0:
                        image = images[0]
                        try:
                            os.mkdir('Thumbnails/{}'.format(experiment.id))
                        except:
                            pass
                        try:
                            os.mkdir('Thumbnails/{}/{}'.format(experiment.id, scan.id))
                        except:
                            pass
                        cv2.imwrite('Thumbnails/{}/{}/thumbnail.jpg'.format(experiment.id, scan.id), image * 255)
                        text = input_image(images, experiment, scan)
                        text_bboxes.append([text, experiment, scan])
                        
                        
                    else:
                        pass#print('No compatible files')
                        
                        
            except xnat.exceptions.XNATResponseError:
                pass
        else:
            pass#print('Incompatible Modality')
    
    project_progress.layout = Layout(display = 'none')
    experiment_progress.layout = Layout(display = 'none')
    experiment
    return text_bboxes

def download_scan(scan, progress = None, all_files = False):
    for file in os.listdir('TempImages'):
        filepath = os.path.join('TempImages', file)
        os.remove(filepath)
                    
    info_text.value = 'Downloading files...'
             
    if progress:
        if all_files:
            progress.max = len(scan.files.values())
        else:
            progress.max = FILES_PER_SCAN - 1
        progress.value = 0
        progress.layout = Layout(display = 'inherit')
        
    ids = []
    
    downloaded = 0
        
    for file_index, file in enumerate(scan.files.values()):
        if progress:
            progress.value = file_index
        if downloaded < FILES_PER_SCAN or all_files:
            c = False
            if 'DICOM_Scrubbed' in scan.resources:
                for scrubbed_file in scan.resources['DICOM_Scrubbed'].files.values():
                    if file.id == scrubbed_file.id:
                        c = True
            if c:
                continue
            file.download(os.path.join('TempImages', str(file_index) + '.dcm'), verbose = False)
            downloaded += 1
            ids.append(file.id)
        else:
            break
            
    return ids
def get_pixel_data(directory):
    progress.max = len(directory) - 1
    progress.value = 0
    progress.layout = Layout(display = 'inherit')
    
    images = []
                    
    info_text.value = 'Extracting pixel data...'
    for counter, file in enumerate(os.listdir(directory)):
        progress.value = counter

        filename = os.path.join(directory, file)
        
        try:
            ds = pydicom.dcmread(filename, force = True)
            image = ds.pixel_array
            
            new_image = image
            
            if len(new_image.shape) > 2:
                new_image = cv2.cvtColor(new_image, cv2.COLOR_BGR2GRAY)
            
            new_image = new_image - np.amin(new_image)
                
            new_image = (new_image) / np.amax(new_image)
            images.append(new_image)
            
        except Exception as e:
            pass
            
    return images
        
            
            
def input_image(images, experiment, scan):
    text = []
    
    #scale = image.shape[0] / 16
    #step_x = int(image.shape[0] / scale)
    #step_y = int(image.shape[1] / scale)
    
    original_shape = images[0].shape
    
    resized_images = []
    for image in images:
        x = 512
        y = int(image.shape[1] / (image.shape[0] / 512))
        resized = np.zeros([x + (x % 32), y + (y % 16)])
        resized[0: y, 0: x] = cv2.resize(image, (x, y))
        resized_images.append(resized)
    
    images = resized_images
    image = images[0]
    
    progress.max = image.shape[0]
    progress.value = 0
    progress.layout = Layout(display = 'inherit')
            
    for x in range(0, image.shape[0], 32):
        progress.value = x
        for y in range(0, image.shape[1], 16):
            preped_images = torch.Tensor([]).to(nn.device)
            for image in images:
                input_image = image[y: y + 16, x: x + 32]
                input_image = cv2.resize(input_image, (128, 64))
                input_image = torch.Tensor(input_image).to(nn.device).view(1, 1, 128, 64)
                preped_images = torch.cat((preped_images, input_image), dim = 0)
                
            preped_images.view(-1, 1, 128, 64).to(nn.device)
            net_out = net(preped_images)
            num_text = 0
            total = 0
            for output in net_out:
                if torch.argmax(output) == 0:
                    num_text += 1
                total += 1
            if float(num_text) / float(total) > TEXT_PCT:
                text.append([x, y])
                
    progress.layout = Layout(display = 'none')
    image = im.fromarray(cv2.merge([image * 255, image * 255, image * 255]).astype(np.uint8))
    image_draw = ImageDraw.Draw(image)
            
    new_text = []
    #for x in range(0, image.width, 32):
     #   for y in range(0, image.height, 16):
            #check = [[x, y] in text, [x - 32, y] in text, [x, y - 16] in text, [x - 32, y - 16] in text]
            #agreed = 0
            #for i in check:
            #    if i:
            #        agreed += 1
            #if agreed >= 3:
    for x, y in text:
        scale = original_shape[0] / 512
        new_text.append([int(x * scale), int(y * scale), int(32 * scale), int(16 * scale)])
        image_draw.rectangle((x + 32, y + 16, x, y), outline = 'red')
                
    text = new_text

    image.save('TempImages/TempImage.jpg')
    image_display.layout = Layout(border_width = '2px', display = 'block')
    image_display.value = open('TempImages/TempImage.jpg', 'rb').read()
    image_display.width = image.width
    image_display.height = image.height
    image_display.format = 'jpg'    
    
    return text

def submit_url():
    global url
            
    url = url_entry.value
    if not '://' in url:
        url = 'https://' + url
        
def submit_project():
    global project
    
    project = project_entry.value
    
def submit_username():
    global auth
    
    auth[0] = username_entry.value
    
def submit_password():
    global auth
    
    auth[1] = password_entry.value
        
        
def start(b):
    global files, good, bad, current_file, current_file_index, project, auth, url, session, current_experiment, current_scan, sproject
    submit_url()
    submit_project()
    submit_username()
    submit_password()
    if url:
        if project:
            if auth[0]:
                if auth[1]:
                    b.disabled = True
                    
                    session = xnat.connect(url, user = auth[0], password = auth[1])
                        
                    sproject = session.projects[project]

                    bboxes = check_project(sproject)
                    report = Report(bboxes)
                    
                    
class Report:
    def edit(self, b):
        if True:
            button, canvas_image, bbox_data, image, has_text = self.report_widgets[self.buttons.index(b)]
            self.current_index = self.buttons.index(b)
            self.current_edit = self.report_widgets[self.buttons.index(b)]
            if len(bbox_data) > 0:
                self.current_has_text = True
            else:
                self.current_has_text = False
            
            save = Button(
                description = 'Save',
                button_style = 'Success',
                icon = 'Save'
            )
            save.on_click(self.save)
            self.report.children = tuple([HBox(children = tuple([save]))])
            
            self.bg_canvas.width = 512
            self.bg_canvas.height = int(image.shape[1]) / (int(image.shape[0]) / 512)
            self.canvas.width = 512
            self.canvas.height = int(image.shape[1]) / (int(image.shape[0]) / 512)
            self.bg_canvas.layout.width = 'auto'
            self.bg_canvas.layout.height = 'auto'#str(int(image.shape[1]) / (int(image.shape[0]) / 512)) + 'px'
            self.canvas.layout.width = 'auto'
            self.canvas.layout.height = 'auto'#.format(int(image.shape[1]) / (int(image.shape[0]) / 512))
            
            self.im_width = self.canvas.width
            self.im_height = self.canvas.height

            self.canvas.on_mouse_down(self.edit_mouse_press)
            self.canvas.on_mouse_up(self.edit_mouse_release)
            
            self.canvas.draw_image(canvas_image, 0, 0, self.canvas.width, self.canvas.height)
            self.canvas.draw_image(self.bg_canvas, 0, 0)

        
    def edit_mouse_move(self, x, y):
        self.mx = x
        self.my = y
        if y < self.bg_canvas.height:
            with hold_canvas(self.canvas):
                self.bg_canvas.clear_rect(0, 0, self.canvas.width, self.bg_canvas.height)
                #self.canvas.draw_image(self.report_widgets[self.current_index][1], 0, 0, self.canvas.width, self.canvas.height)
                self.bg_canvas.stroke_style = 'green'
                self.bg_canvas.line_width = 2
                self.bg_canvas.stroke_rect((self.mx // 32) * 32 - 2, (self.my // 16) * 16 - 2, 32 + 4, 16 + 4)
        
    def edit_mouse_press(self, x, y):
        self.mx, self.my = x, y
        
    def edit_mouse_release(self, x, y):
        button, canvas_image, bbox_data, image, has_text = self.current_edit
        ax, ay = (self.mx // 32) * 32, (self.my // 16) * 16
        scale = (int(image.shape[0]) / 512)
        osx, osy, osw, osh = int(ax * scale), int(ay * scale), int(32 * scale), int(16 * scale)
        
        w, h = int(abs(self.mx - x)), int(abs(self.my - y))
        self.mx, self.my = int(min(x, self.mx)), int(min(self.my, y))
        
        if [osx, osy, osw, osh] in bbox_data[0]:
            remove = True
        else:
            remove = False
        
        for x in range(self.mx, self.mx + w + 16, 32):
            for y in range(self.my, self.my + h + 8, 16):
                ax, ay = (x // 32) * 32, (y // 16) * 16
                sx, sy, sw, sh = int(ax * scale), int(ay * scale), int(32 * scale), int(16 * scale)
                if remove:
                    if [sx, sy, sw, sh] in bbox_data[0]:# not self.saving:
                        self.current_edit[2][0].remove([sx, sy, sw, sh])
                else:
                    if not [sx, sy, sw, sh] in bbox_data[0]:#not self.saving:
                        self.current_edit[2][0].append([sx, sy, sw, sh])
        
        pil_image = im.fromarray((image).astype(np.uint8))
        image_draw = ImageDraw.Draw(pil_image)

        for bbox in self.current_edit[2][0]:
            image_draw.rectangle((bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1] + bbox[3]), outline = 'red', width = 2)
        pil_image.save('TempImages/0.jpg')
        self.report_widgets[self.current_index][1] = Image(
            value = open('TempImages/0.jpg', 'rb').read(),
            format = 'jpg',
            width = 512,
            height = pil_image.height / (pil_image.width / 512),
            layout = Layout(margin = '5px')
        )
                        #self.bg_canvas.clear_rect(0, 0, self.bg_canvas.width, self.bg_canvas.height)
                        #self.bg_canvas.draw_image(Image.from_file('TempImages/0.jpg'), 0, 0, 512, int(canvas_image.height) / (int(canvas_image.width) / 512))
        with hold_canvas(self.canvas):
            self.canvas.clear_rect(0, 0, self.canvas.width, self.bg_canvas.height)
            self.canvas.draw_image(self.report_widgets[self.current_index][1], 0, 0, self.canvas.width, self.canvas.height)

                
            
    def save(self, b):
        self.report_widgets[self.current_index] = self.current_edit
        image = self.report_widgets[self.current_index][3]
        pil_image = im.fromarray((image).astype(np.uint8))
        image_draw = ImageDraw.Draw(pil_image)

        for bbox in self.report_widgets[self.current_index][2][0]:
            image_draw.rectangle((bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1] + bbox[3]), outline = 'red', width = 2)
        pil_image.save('TempImages/0.jpg')
        self.report_widgets[self.current_index][1] = Image(
            value = open('TempImages/0.jpg', 'rb').read(),
            format = 'jpg',
            width = 512,
            height = pil_image.height / (pil_image.width / 512),
            layout = Layout(margin = '5px')
        )
        
        if self.report_widgets[self.current_index][4]:
            l = list(self.images[self.current_page_index].children)
            l[1] = self.report_widgets[self.current_index][1]
            self.images[self.current_page_index].children = l
        else:
            l = list(self.non_images[self.current_page_non_index].children)
            l[1] = self.report_widgets[self.current_index][1]
            self.non_images[self.current_page_non_index].children = l
        
        self.report.children = ()
        self.bg_canvas.width = 0
        self.bg_canvas.height = 0
        self.canvas.width = 0
        self.canvas.height = 0
        self.report.children = (VBox(children = tuple([self.ya_label, self.current_page, self.non_label, self.current_page_non])), self.save_box)
        
    
    def filesave(self, b):
        try:
            os.mkdir('Scrubbed Dicoms')
        except FileExistsError:    
            for root, dirs, files in os.walk("Scrubbed Dicoms"):
                for name in files:
                    os.remove(os.path.join(root, name))

        
        p1 = IntProgress(description = 'Scan: ')
        p2 = IntProgress()
        
        self.report.children = self.report.children + (p1, p2)
        
        p1.min = 0
        p2.min = 0
        
        p1.value = 0
        p2.value = 0
        
        p1.max = len(self.bboxes)
        
        for bbox_data in self.bboxes:
            p1.value = self.bboxes.index(bbox_data) + 1
            
            for file in os.listdir('TempImages'):
                filepath = os.path.join('TempImages', file)
                os.remove(filepath)
            
            p2.description = 'Download: '
            ids = download_scan(bbox_data[2], progress = p2, all_files = True)
            
            try:
                os.mkdir('Scrubbed Dicoms/' + bbox_data[1].id)
                os.mkdir('Scrubbed Dicoms/' + bbox_data[1].id + '/' + bbox_data[2].id)
            except FileExistsError as e:
                print('File Exists: {}'.format(e))
                
            
            
            p2.max = len(os.listdir('TempImages'))
            p2.value = 0
            p2.description = 'Scrubbing: '
            
            for count, file in enumerate(os.listdir('TempImages')):
                p2.value = count + 1
                
                filename = 'TempImages/' + file
                try:
                    ds = pydicom.dcmread(filename, force = True)
                    image = ds.pixel_array
                    

                    for bbox in bbox_data[0]:
                        image[bbox[1]: bbox[1] + bbox[3], bbox[0]: bbox[0] + bbox[2]] = np.zeros((bbox[3], bbox[2], image.shape[2]))

                    ds.PixelData = image.tobytes()
                        
                    ds.save_as('Scrubbed Dicoms/{}/{}/{}'.format(bbox_data[1].id, bbox_data[2].id, 'scrubbed_' + ids[count]))
                except Exception as e:
                    print('Real Error: ', e)
                    
    def xnatupload(self, b):
        try:
            os.mkdir('Scrubbed Dicoms')
        except FileExistsError:    
            for root, dirs, files in os.walk("Scrubbed Dicoms"):
                for name in files:
                    os.remove(os.path.join(root, name))

        
        p1 = IntProgress(description = 'Scan: ')
        p2 = IntProgress()
        
        self.report.children = self.report.children + (p1, p2)
        
        p1.min = 0
        p2.min = 0
        
        p1.value = 0
        p2.value = 0
        
        p1.max = len(self.bboxes)
        
        for bbox_data in self.bboxes:
            p1.value = self.bboxes.index(bbox_data) + 1
            
            for file in os.listdir('TempImages'):
                filepath = os.path.join('TempImages', file)
                os.remove(filepath)
            
            if len(bbox_data[0]) > 0:
                p2.description = 'Download: '
                ids = download_scan(bbox_data[2], progress = p2, all_files = True)

                try:
                    os.mkdir('Scrubbed Dicoms/' + bbox_data[1].id)
                    os.mkdir('Scrubbed Dicoms/' + bbox_data[1].id + '/' + bbox_data[2].id)
                except FileExistsError as e:
                    print('File Exists: {}'.format(e))
                
            
                try:
                    bbox_data[2].create_resource(label = 'DICOM_Scrubbed', format='DICOM')
                except:
                    pass
                
                resource = bbox_data[2].resources['DICOM_Scrubbed']
                #zip_obj = ZipFile('Scrubbed Dicoms/zip.zip', 'w')
                
                p2.max = len(os.listdir('TempImages'))
                p2.value = 0
                p2.description = 'Scrubbing: '

                for count, file in enumerate(os.listdir('TempImages')):
                    p2.value = count + 1

                    filename = 'TempImages/' + file
                    try:
                        ds = pydicom.dcmread(filename, force = True)
                        image = ds.pixel_array


                        for bbox in bbox_data[0]:
                            image[bbox[1]: bbox[1] + bbox[3], bbox[0]: bbox[0] + bbox[2]] = np.zeros((bbox[3], bbox[2], image.shape[2]))

                        ds.PixelData = image.tobytes()

                        ds.save_as('Scrubbed Dicoms/{}/{}/{}'.format(bbox_data[1].id, bbox_data[2].id, 'scrubbed_' + ids[count]))
                        #zip_obj.write('Scrubbed Dicoms/{}/{}/{}'.format(bbox_data[1].id, bbox_data[2].id, 'scrubbed_' + ids[count]), arcname = ids[count])
                        #resource.upload('Scrubbed Dicoms/{}/{}/{}'.format(bbox_data[1].id, bbox_data[2].id, file), file)

                    except Exception as e:
                        print('Real Error: ', e)
                
                zip_obj.close()
                resource.upload_dir('Scrubbed Dicoms/{}/{}'.format(bbox_data[1].id, bbox_data[2].id))
                
    def nextpage(self, b):
        if self.current_page_index + 1 <= len(self.images) - 1:
            self.current_page_index += 1
            self.current_page = self.images[self.current_page_index]
            self.report.children = (VBox(children = tuple([self.ya_label, self.current_page, self.non_label, self.current_page_non])), self.save_box)
            
    def lastpage(self, b):
        if self.current_page_index > 0:
            self.current_page_index -= 1
            self.current_page = self.images[self.current_page_index]
            self.report.children = (VBox(children = tuple([self.ya_label, self.current_page, self.non_label, self.current_page_non])), self.save_box)         
            
    def non_nextpage(self, b):
        if self.current_page_non_index + 1 <= len(self.non_images) - 1:
            self.current_page_non_index += 1
            self.current_page_non = self.non_images[self.current_page_non_index]
            self.report.children = (VBox(children = tuple([self.ya_label, self.current_page, self.non_label, self.current_page_non])), self.save_box)
            
    def non_lastpage(self, b):
        if self.current_page_non_index > 0:
            self.current_page_non_index -= 1
            self.current_page_non = self.non_images[self.current_page_non_index]
            self.report.children = (VBox(children = tuple([self.ya_label, self.current_page, self.non_label, self.current_page_non])), self.save_box)

            
    def generate_report(self):
        for bbox_data in self.bboxes:
            for file in os.listdir('TempImages'):
                filepath = os.path.join('TempImages', file)
                os.remove(filepath)

            image = cv2.imread('Thumbnails/{}/{}/thumbnail.jpg'.format(bbox_data[1].id, bbox_data[2].id), cv2.IMREAD_COLOR)

            pil_image = im.fromarray((image).astype(np.uint8))
            image_draw = ImageDraw.Draw(pil_image)
            
            for bbox in bbox_data[0]:
                image_draw.rectangle((bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1] + bbox[3]), outline = 'red', width = 2)
            pil_image.save('TempImages/0.jpg')
            
            canvas_image = Image(
                value = open('TempImages/0.jpg', 'rb').read(),
                format = 'jpg',
                width = 512,
                height = pil_image.height / (pil_image.width / 512),
                layout = Layout(margin = '5px')
            )

            button = Button(
                description = ' Edit',
                button_style = 'info',
                icon = 'edit',
                layout = Layout(margin = '5px')
            )
            
            button.on_click(self.edit)
            
            nextpage_button = Button(
                    icon = 'caret-right',
                    button_style = 'warning',
                    layout = Layout(width = '100px', margin='5px 25px 5px 5px')
            )
            lastpage_button = Button(
                    icon = 'caret-left',
                    button_style = 'warning',
                    layout = Layout(width = '100px', margin='5px 5px 5px 25px')
            )
            
            
            if len(bbox_data[0]) > 0:
                label = Label(value = '{} of {}'.format(self.ya_bboxes.index(bbox_data) + 1, len(self.ya_bboxes)), layout = Layout())
                nextpage_button.on_click(self.nextpage)
                lastpage_button.on_click(self.lastpage)
                self.images.append(VBox(
                    children = (Box(), canvas_image, HBox(children=(button, lastpage_button, nextpage_button, label)), Box()),
                    layout = Layout(margin = '10px 10px 10px 10px', padding = '5px 5px 5px 5px', border = '1px solid', width = '532px')
                ))
                t = True
            else:
                label = Label(value = '{} of {}'.format(self.non_bboxes.index(bbox_data) + 1, len(self.non_bboxes)), layout = Layout())
                nextpage_button.on_click(self.non_nextpage)
                lastpage_button.on_click(self.non_lastpage)
                self.non_images.append(VBox(
                    children = (Box(), canvas_image, HBox(children=(button, lastpage_button, nextpage_button, label)), Box()),
                    layout = Layout(margin = '10px 10px 10px 10px', padding = '5px 5px 5px 5px', border = '1px solid', width = '532px')
                ))
                t = False

            self.report_widgets.append([button, canvas_image, bbox_data, image, t])
            self.buttons.append(button)
        
        self.filesave_button = Button(
                description = 'Save as Files',
                button_style = 'Success',
                icon = 'download',
                layout = Layout(grid_column = '2', margin = '5px')
                
        )
            
        self.filesave_button.on_click(self.filesave)
            
        self.xnatsave_button = Button(
                description = 'Upload to XNAT',
                button_style = 'Info',
                icon = 'upload',
                layout = Layout(grid_column ='3', margin = '5px', height = 'auto')
        )
            
        self.xnatsave_button.on_click(self.xnatupload)
            
        self.save_box = GridBox(children = (self.filesave_button, self.xnatsave_button),
                                layout = Layout(margin = '15px', width = '532px', display = 'grid', grid_template_columns = '1fr 1fr 1fr 1fr', grid_template_rows = '1fr'))
        
        if len(self.images) > 0:
            self.current_page = self.images[self.current_page_index]
            self.ya_label = HTML(value='<H1>Images With Text</H1>')
        else:
            self.ya_label = Box()
            self.current_page = Box()
        if len(self.non_images) > 0:
            self.current_page_non = self.non_images[self.current_page_non_index]
            self.non_label = HTML(value='<H1>Images Without Text</H1>')
        else:
            self.current_page_non = Box()
            self.non_label = Box()
            
        self.report = VBox(
                children = (VBox(children = tuple([self.ya_label, self.current_page, self.non_label, self.current_page_non])), self.save_box)
            )
    
    def __init__(self, bboxes):
        app.close()
        
        self.bboxes = bboxes

        self.images = []
        self.non_images = []
        self.report_widgets = []
        self.buttons = []
        
        self.non_bboxes = []
        self.ya_bboxes = []
        for i in self.bboxes:
            if len(i[0]) > 0:
                self.ya_bboxes.append(i)
            else:
                self.non_bboxes.append(i)
        
        self.im_width = None
        self.im_height = None
        
        self.current_edit = None
        self.current_index = None
        self.current_has_text = None
        
        self.current_page = None
        self.current_page_index = 0
        
        self.current_page_non = None
        self.current_page_non_index = 0
        
        self.report = None
        
        self.saving = False
        
        self.mx = 0
        self.my = 0
        
        self.bg_canvas = Canvas(width = 1, height = 1)
        self.canvas = Canvas(width = 1, height = 1)
        
        self.generate_report()
        
        display(self.canvas)
        display(self.report)
    
url = None
project = None
auth = [None, None]

report_widgets =  []

FILES_PER_SCAN = 10
TEXT_PCT = .50

url_entry = Text(
    value = '',
    placeholder = 'ex: https://xnat-demo.radiologics.com',
    description = 'URL:',
)

project_entry = Text(
    value = '',
    placeholder = '',
    description = 'Project:',
)

username_entry = Text(
    value = '',
    placeholder = '',
    description = 'Username: '
)

password_entry = Password(
    value = '',
    placeholder = '',
    description = 'Password:',
)

entry_frame = VBox(
    children = (url_entry, project_entry, username_entry, password_entry),
)

start_button = Button(
    description = ' Start',
    button_style = 'success',
    icon = 'play'
)

start_button.on_click(start)

info_text = Label(
    layout = Layout(padding = '0 0 0 25px')
)

start_info_frame = HBox(
    children = (start_button, info_text),
    layout = Layout(padding = '10px 10px 10px 10px')
)

experiment_progress = IntProgress(
    value = 0,
    min = 0,
    max = 0,
    step = 1, 
    description = 'Experiment: ',
    orientation = 'horizontal',
    layout = Layout(display = 'none')
)
project_progress = IntProgress(
    value = 0,
    min = 0,
    max = 0,
    step = 1, 
    description = 'Project: ',
    orientation = 'horizontal',
    layout = Layout(display = 'none')
)

files_info = IntProgress(
    value = 0,
    min = 0, 
    max = 0,
    step = 1, 
    description = 'Files: ',
    orientation = 'horizontal',
    layout = Layout(display = 'none')
)

progress = IntProgress(
    orientation = 'horizontal',
    layout = Layout(display = 'none')
)

image_display = Image(
    layout = Layout(border_width = '2px', display = 'none')
)

app = VBox(
    children = (entry_frame, start_info_frame, progress, image_display, experiment_progress, project_progress)
)
display(app)

Canvas(height=1, width=1)